In [1]:
%matplotlib inline 

In [2]:
from time import time
from glob import glob

from torchvision.io import read_image
from torchvision import datasets, transforms
from matplotlib import pyplot as plt

import numpy as np
from sklearn.model_selection import KFold, StratifiedKFold

from tqdm import tqdm
import torchmetrics
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, Dataset, SubsetRandomSampler, ConcatDataset
plt.gray()

<Figure size 640x480 with 0 Axes>

In [3]:
batch_size = 64
input_size = 28
num_classes = 47

In [5]:
# transform = transforms.Compose([transforms.Resize((input_size, input_size)), transforms.ToTensor(), transforms.Normalize(mean=(0.5,), std=(0.5,))])
datasets.ImageFolder
# data_loader = DataLoader(
#         datasets.EMNIST('data/emnist', train=True, download=True, transform=transform, split="bymerge"),
#         batch_size=batch_size, shuffle=True)
# data, label = data_loader.__iter__().__next__()
# label

tensor([39, 15, 46,  1, 23,  0,  1, 36, 25,  1, 18,  5, 24,  3,  9, 21,  7, 38,
         9,  8,  5,  2,  0,  8, 30,  2, 32, 35, 45, 22, 30,  8, 46, 23,  8, 38,
         9,  8, 27, 44,  4, 45, 29, 45, 45,  8, 18,  2,  4,  9, 32,  6, 22,  8,
        20,  4,  2, 39, 33,  9, 22,  8,  0, 34])

In [6]:
# plt.imshow(data[2, 0, :] * 255, interpolation='nearest')
counter = [0] * num_classes
for _, l in data_loader:
    for v in l.numpy():
        counter[v] += 1

In [9]:
counter = np.array(counter)
hist = counter[:47] / (counter.sum()) * 100
hist

array([4.96008207, 5.48821375, 4.91552186, 5.05565012, 4.82224629,
       4.48181198, 4.89302683, 5.16096124, 4.86064545, 4.85462767,
       0.91857086, 0.5550684 , 1.85734427, 0.65994968, 0.70565614,
       1.30356539, 0.36307262, 0.4437395 , 2.11095064, 0.81512239,
       0.71611561, 2.92019853, 1.6637724 , 1.18020094, 3.96370993,
       1.53997811, 0.37295897, 0.72313635, 3.36837973, 1.39927672,
       2.20479932, 1.08721193, 1.06070505, 0.80208387, 1.01614484,
       0.77600683, 1.43409387, 0.72786461, 1.45458297, 3.53286567,
       0.3632159 , 0.52913464, 1.24396073, 1.63970129, 0.42496977,
       2.0145229 , 2.61458136])

In [4]:
class LeNet(nn.Module):

  def __init__(self, num_classes=10, device="cuda"):
    super(LeNet, self).__init__()
    self.num_classes = num_classes
    self.device = device

    self.core = nn.Sequential(
      nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1, padding=0),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride=2),

      nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=0),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride=2),

      nn.Flatten(start_dim=1),

      nn.Linear(16*4*4, 120),
      nn.ReLU(),
      nn.Linear(120, 84),
      nn.ReLU(),
      nn.Linear(84, self.num_classes),
      nn.Softmax(dim=1),
    )
    self.core = self.core.to(self.device)
    self.to(self.device)

  def forward(self, x):
    res = self.core(x)
    return res


class ANN(nn.Module):
  def __init__(self, num_classes=10, device="cuda"):
    super(ANN, self).__init__()
    self.num_classes = num_classes
    self.device = device
    self.core = nn.Sequential(
        nn.Flatten(),
        nn.Linear(28*28, 300),
        nn.ReLU(),
        nn.Linear(300, 100),
        nn.ReLU(),
        nn.Linear(100, self.num_classes),
        nn.Softmax(dim=1),
    )

  def forward(self, x):
    return self.core(x)


class IDK(nn.Module):
  def __init__(self):
    super().__init__()
    self.network = nn.Sequential(
      nn.Conv2d(1, 32, kernel_size = 3, padding = 1),
      nn.ReLU(),
      nn.Conv2d(32,64, kernel_size = 3, stride = 1, padding = 1),
      nn.ReLU(),
      nn.MaxPool2d(2,2),
  
      nn.Conv2d(64, 128, kernel_size = 3, stride = 1, padding = 1),
      nn.ReLU(),
      nn.Conv2d(128 ,128, kernel_size = 3, stride = 1, padding = 1),
      nn.ReLU(),
      nn.MaxPool2d(2,2),
      
      nn.Conv2d(128, 256, kernel_size = 3, stride = 1, padding = 1),
      nn.ReLU(),
      nn.Conv2d(256,256, kernel_size = 3, stride = 1, padding = 1),
      nn.ReLU(),
      nn.MaxPool2d(2,2),
      
      nn.Flatten(),
      nn.Linear(82944,1024),
      nn.ReLU(),
      nn.Linear(1024, 512),
      nn.ReLU(),
      nn.Linear(512,6)
    )
  
  def forward(self, xb):
    return self.network(xb)

In [32]:
IGNORED_PARAM_RESET = {"activation", "loss_function"}
def reset_modules(module, parents_modules_names=[]):
  for name, module in module.named_children():
    if name in IGNORED_PARAM_RESET:
      continue
    if isinstance(module, nn.ModuleList):
        reset_modules(module, parents_modules_names=[*parents_modules_names, name])
    elif isinstance(module, nn.Dropout):
      continue
    else:
      module.reset_parameters()

def score(prediction, target, num_classes=10, device="cuda"):
  accuracy = torchmetrics.Accuracy("multiclass", num_classes=num_classes, average="macro").to(device)
  recall = torchmetrics.Recall("multiclass", num_classes=num_classes, average="macro").to(device)
  precision = torchmetrics.Precision("multiclass", num_classes=num_classes, average="macro").to(device)

  return accuracy(prediction, target), recall(prediction, target), precision(prediction, target)

def forward(model, x):
  return model(x)


def test(model, test_dataset, num_classes, device):
  test_dataloader = DataLoader(test_dataset, batch_size=64)
  all_preds = []
  all_targets = []

  model.eval()
  with torch.no_grad():
    for X, y in tqdm(test_dataloader, leave=False):
      X = X.to(device)
      y = y.to(device)
      pred = forward(model, X)
      all_preds.extend(torch.sigmoid(pred))
      all_targets.extend(y)
    all_preds = torch.stack(all_preds)
    all_targets = torch.stack(all_targets)
  return score(all_preds, all_targets, num_classes, device)

def learn(train_dataset, test_dataset, get_model, num_classes=10, lr=0.0005, n_splits=3, n_epoch=5, batch_size=64, device="cuda"):
  kfolder = KFold(n_splits=n_splits, shuffle=True)
  splits = [None] * n_splits
  for i, (train_ids, label_ids) in enumerate(kfolder.split(train_dataset.data)):
    splits[i] = (train_ids, label_ids)

  scheduler_args = {"verbose": True, "min_lr":1e-9, "threshold": 20, "cooldown": 5, "patience": 20, "factor":0.25, "mode": "min"}
  metrics = [None] * n_splits
  loss_function = nn.CrossEntropyLoss()

  for fold, (train_ids, validation_ids) in enumerate(splits):

    train_subsampler = SubsetRandomSampler(train_ids)
    validation_subsampler = SubsetRandomSampler(validation_ids)
    train_loader = DataLoader(train_dataset, batch_size=batch_size,
                                    sampler=train_subsampler)
    validation_loader = DataLoader(train_dataset, batch_size=128,
                                    sampler=validation_subsampler)
    model = get_model()
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = ReduceLROnPlateau(optimizer, **scheduler_args)
    model.to(device)

    total_loss = []
    total_validation_loss = []
    for e in range(n_epoch):
      model.train()
      loss = 0
      epoch_loss = 0
      epoch_start_time = time()
      for (X, y) in tqdm(train_loader, leave=False):
        optimizer.zero_grad()
        X = X.to(device)
        y = y.to(device)

        y_hat = forward(model, X)
        loss = loss_function(y_hat, y)
        loss.backward()

        optimizer.step()
        epoch_loss += loss.item()
      
      epoch_loss /= len(train_ids)
      total_loss.append(epoch_loss)
      scheduler.step(loss)

      all_preds = []
      all_targets = []
      validation_loss = 0
      model.eval()
      with torch.no_grad():
        for X, y in tqdm(validation_loader, leave=False):
          X = X.to(device)
          y = y.to(device)

          pred = model.forward(X)
          loss = loss_function(pred, y)
          validation_loss += loss.item()

          all_preds.extend(torch.sigmoid(pred) if isinstance(loss_function, nn.BCEWithLogitsLoss) else pred)
          all_targets.extend(y)
        validation_loss /= len(validation_ids)
        total_validation_loss.append(validation_loss)
      
      all_preds = torch.stack(all_preds)
      all_targets = torch.stack(all_targets)
      accuracy_value, recall_value, precision_value, *_ = score(all_preds, all_targets, num_classes, device)

      print(f"fold: {fold} | epoch: {e} @ {(time() - epoch_start_time):>0.1f}s | train -> loss: {(epoch_loss):>0.5f} | validation -> loss: {(validation_loss):>0.5f} | accuracy: {(100 * accuracy_value):>0.6f} | precision: {(100 * precision_value):>0.6f} | recall: {(100 * recall_value):>0.6f}")
    accuracy_value, recall_value, precision_value, *_ = test(model, test_dataset, num_classes, device)
    print(f"fold: {fold} on Test Set | accuracy: {(100 * accuracy_value):>0.6f} | precision: {(100 * precision_value):>0.6f} | recall: {(100 * recall_value):>0.6f}")
    metrics[fold] = (accuracy_value, recall_value, precision_value)
  return metrics

In [29]:
transform = transforms.Compose([transforms.Resize((input_size, input_size)), transforms.ToTensor(), transforms.Normalize(mean=(0.5,), std=(0.5,))])
num_classes = 47
train_set = datasets.EMNIST('data/emnist', train=True, download=True, transform=transform, split="bymerge")
test_set = datasets.EMNIST('data/emnist', train=False, download=True, transform=transform, split="bymerge")
get_model = lambda: LeNet(num_classes)
torch.cuda.empty_cache()
metrics1 = learn(train_set, test_set, get_model, lr=0.0001, num_classes=num_classes, n_splits=5, n_epoch=15, batch_size=128)

fold: 0 | epoch: 0 | train -> loss: 0.02735 | validation -> loss: 0.02702 | accuracy: 18.341988 | precision: 9.122836 | recall: 18.341988


fold: 0 | epoch: 1 | train -> loss: 0.02697 | validation -> loss: 0.02696 | accuracy: 18.587402 | precision: 8.992884 | recall: 18.587402


fold: 0 | epoch: 2 | train -> loss: 0.02668 | validation -> loss: 0.02654 | accuracy: 21.313921 | precision: 12.275155 | recall: 21.313921


fold: 0 | epoch: 3 | train -> loss: 0.02650 | validation -> loss: 0.02651 | accuracy: 21.514383 | precision: 12.128469 | recall: 21.514383


fold: 0 | epoch: 4 | train -> loss: 0.02648 | validation -> loss: 0.02650 | accuracy: 21.599560 | precision: 12.265759 | recall: 21.599560


...

fold: 4 | epoch: 11 | train -> loss: 0.02570 | validation -> loss: 0.02570 | accuracy: 32.317917 | precision: 21.300310 | recall: 32.317917


fold: 4 | epoch: 12 | train -> loss: 0.02570 | validation -> loss: 0.02570 | accuracy: 32.239120 | precision: 21.506432 | recall: 32.239120


fold: 4 | epoch: 13 | train -> loss: 0.02569 | validation -> loss: 0.02569 | accuracy: 32.352371 | precision: 21.273664 | recall: 32.352371


fold: 4 | epoch: 14 | train -> loss: 0.02569 | validation -> loss: 0.02568 | accuracy: 32.381840 | precision: 21.217213 | recall: 32.381840


fold: 4 on Test Set | accuracy: 32.419716 | precision: 21.245306 | recall: 32.419716


In [39]:
torch.cuda.empty_cache() # 203.9s 
transform = transforms.Compose([transforms.Resize((input_size, input_size)), transforms.ToTensor(), transforms.Normalize(mean=(0.5,), std=(0.5,))])
num_classes = 47
# train_set = datasets.EMNIST('data/emnist', train=True, download=True, transform=transform, split="bymerge")
test_set = datasets.EMNIST('data/emnist', train=False, download=True, transform=transform, split="bymerge")
get_model = lambda: ANN(num_classes)
metrics = learn(train_set, test_set, get_model, lr=0.0001, num_classes=num_classes, n_splits=5, n_epoch=15, batch_size=128)

fold: 0 | epoch: 0 @ 208.0s | train -> loss: 0.02678 | validation -> loss: 0.02642 | accuracy: 22.714008 | precision: 13.549365 | recall: 22.714008


fold: 0 | epoch: 1 @ 207.3s | train -> loss: 0.02634 | validation -> loss: 0.02630 | accuracy: 24.691383 | precision: 14.776370 | recall: 24.691383


fold: 0 | epoch: 2 @ 230.1s | train -> loss: 0.02605 | validation -> loss: 0.02595 | accuracy: 30.613312 | precision: 18.663040 | recall: 30.613312


fold: 0 | epoch: 3 @ 238.1s | train -> loss: 0.02579 | validation -> loss: 0.02574 | accuracy: 36.713905 | precision: 23.077812 | recall: 36.713905


fold: 0 | epoch: 4 @ 236.7s | train -> loss: 0.02558 | validation -> loss: 0.02544 | accuracy: 40.911674 | precision: 27.811420 | recall: 40.911674


...

fold: 4 | epoch: 11 @ 207.4s | train -> loss: 0.02532 | validation -> loss: 0.02533 | accuracy: 40.395840 | precision: 26.991066 | recall: 40.395840


fold: 4 | epoch: 12 @ 211.2s | train -> loss: 0.02531 | validation -> loss: 0.02533 | accuracy: 40.437752 | precision: 27.299591 | recall: 40.437752


fold: 4 | epoch: 13 @ 214.6s | train -> loss: 0.02530 | validation -> loss: 0.02532 | accuracy: 40.651974 | precision: 27.162842 | recall: 40.651974


fold: 4 | epoch: 14 @ 233.5s | train -> loss: 0.02529 | validation -> loss: 0.02532 | accuracy: 40.548527 | precision: 27.195677 | recall: 40.548527


fold: 4 on Test Set | accuracy: 40.543793 | precision: 27.072071 | recall: 40.543793


In [87]:
class CustomImages(Dataset):
    
    def __init__(self, root, transform=lambda x: x):
        self.transform = transform
        if len(root) == 0:
            raise ValueError()
        targets = []
        data = []
        for folder in tqdm(glob(root + "*")):
            label = int(folder.split("\\")[-1])
            files = glob(folder + "\\*")
            ims = [None] * len(files)
            for i, f in tqdm(enumerate(files), leave=False, desc=f"Label '{label}'"):
                ims[i] = read_image(f)[0]
            targets.extend([label]*len(files))
            data.extend(ims)

        self.targets = torch.tensor(targets)
        self.data = torch.stack(data)
        self.data = self.data.reshape(-1, 1, *self.data.shape[1:]).to(torch.float)
    
    def __len__(self) -> int:
        return len(self.data)

    def __getitem__(self, index: int):
        return self.transform(self.data[index]), self.targets[index]

In [88]:
torch.cuda.empty_cache() # 414
transform = transforms.Compose([transforms.Resize((input_size, input_size), antialias=None), transforms.Normalize(mean=(0.5,), std=(0.5,))])
transform_test = transforms.Compose([transforms.Resize((input_size, input_size), antialias=None), transforms.ToTensor(), transforms.Normalize(mean=(0.5,), std=(0.5,))])
train_set = CustomImages("data\\augmented\\acgan\\", transform=transform)
print("loaded train set")
test_set = datasets.EMNIST('data/emnist', train=False, download=True, transform=transform_test, split="bymerge")
print("loaded test set")
get_model = lambda: LeNet(num_classes)

metrics_cgan = learn(train_set, test_set, get_model, lr=0.0001, num_classes=num_classes, n_splits=5, n_epoch=15, batch_size=128)

 62%|██████▏   | 29/47 [03:08<04:04, 13.59s/it]